# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686586


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:30,  3.24s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:59,  2.29s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:41,  1.65s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:27,  1.20s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:10<00:00,  5.90it/s]

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  5.35it/s]

Rendering models:  93%|█████████▎| 28/30 [00:10<00:00,  4.51it/s]

Rendering models: 100%|██████████| 30/30 [00:10<00:00,  5.45it/s]

not-logged-in-f6fac0763296d5759c01    0.018774
Bredit                                0.000529
asucich                               0.000355
thedarktree                           0.000240
not-logged-in-ff40b895d06514783cbc    0.001760
kcarte13                              0.000533
jmfranci                              0.000520
not-logged-in-22147dc175d9d5a49da1    0.010560
ElisabethB                            0.049879
wrxtminecrafter                       0.000957
not-logged-in-4d75098d04b85e5c847c    0.004938
revolcka                              0.000383
not-logged-in-468edef11712cfc1e24c    0.007902
ComeMyChild                           0.001535
clairedeu                             0.000334
acapirala                             0.000279
arad98                                0.017357
pangeli5                              0.000154
dinosora                              0.000266
Lavadude                              0.024573
fanboy_deluxe                         0.004140
asenkow      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())